In [1]:
#All combinations

import sgkit as sg
import pandas as pd
import numpy as np
import zarr
import xarray as xr
import allel

In [2]:
#2R analysis

ds2 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2R.zarr')
ds2

ds2['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds2['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds2['sample_cohort_name']

sample_is_hyb_2R = ds2['sample_cohort_name'] == 'hyb'

sample_is_ara_2R = ds2['sample_cohort_name'] == 'ara'

sample_is_gam_2R = ds2['sample_cohort_name'] == 'gam'

g_2R = allel.GenotypeArray(ds2['call_genotype'])

In [12]:
import pyranges as pr

# import annotations
ann_db_2R = pr.read_gff3("/Users/christinagkertsou/Desktop/Finalyearproject/VectorBase-56_AgambiaePEST.gff")
ann_db_2R = ann_db_2R['AgamP4_2R'] 
ann_db_2R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_2R,VEuPathDB,protein_coding_gene,24245442,24251444,.,+,.,AGAP002619,unspecified product,NaN,NaN,NaN,NaN,NaN
1,AgamP4_2R,VEuPathDB,mRNA,24245442,24251444,.,+,.,AGAP002619-RA,unspecified product,AGAP002619,NaN,NaN,NaN,NaN
2,AgamP4_2R,VEuPathDB,exon,24245442,24245526,.,+,.,exon_AGAP002619-RA-E1,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN
3,AgamP4_2R,VEuPathDB,exon,24245644,24246259,.,+,.,exon_AGAP002619-RA-E2,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN
4,AgamP4_2R,VEuPathDB,exon,24247851,24249256,.,+,.,exon_AGAP002619-RA-E3,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53185,AgamP4_2R,VEuPathDB,exon,47020993,47023594,.,-,.,exon_AGAP029680-RA-E1,NaN,AGAP029680-RA,AGAP029680,NaN,NaN,NaN
53186,AgamP4_2R,VEuPathDB,CDS,47018304,47020467,.,-,0,AGAP029680-PA-CDS2,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN
53187,AgamP4_2R,VEuPathDB,CDS,47020993,47021209,.,-,0,AGAP029680-PA-CDS1,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN
53188,AgamP4_2R,VEuPathDB,three_prime_UTR,47017982,47018304,.,-,.,utr_AGAP029680-RA_1,NaN,AGAP029680-RA,NaN,NaN,NaN,NaN


In [13]:
#ara preparation

is_ara_missing_2R = g_2R[:,sample_is_ara_2R].is_missing().all(axis=1)

# is_ara_hom_alt = g[:,sample_is_ara].is_hom_alt().sum(axis=1) > 2
is_ara_hom_alt_2R = g_2R[:,sample_is_ara_2R].is_hom_alt().all(axis=1)
is_ara_hom_ref_2R = g_2R[:,sample_is_ara_2R].is_hom_ref().sum(axis=1) == 2

In [14]:
#gam preparation

is_gam_missing_2R = g_2R[:,sample_is_gam_2R].is_missing().sum(axis=1) > 6
is_gam_hom_alt_2R = g_2R[:,sample_is_gam_2R].is_hom_alt().sum(axis=1) > 6
is_gam_hom_ref_2R = g_2R[:,sample_is_gam_2R].is_hom_ref().sum(axis=1) > 6

In [15]:
sum(is_gam_hom_ref_2R)

192540

In [7]:
#hyribrids hets

hyb_het_2R = g_2R[:,sample_is_hyb_2R].is_het().sum(axis=1) > 2

In [16]:
#The ara and gam fixed masks

mask_ara_fixed_2R = is_ara_hom_alt_2R & (is_gam_missing_2R | is_gam_hom_ref_2R)

mask_gam_fixed_2R = is_gam_hom_alt_2R & (is_ara_missing_2R | is_ara_hom_ref_2R)

In [17]:
#Scenario 1: het hybs with mask_gam_fixed_2R

mask_hets_2R=hyb_het_2R & mask_gam_fixed_2R

In [18]:
#positions

pos_2R = allel.SortedIndex(ds2['variant_position'])

pos_gam_fixed_2R = pos_2R[mask_gam_fixed_2R]

pos_hyb_hets_2R = pos_2R[mask_hets_2R]

In [19]:
import pyranges as pr

pr_gam_fixed_2R = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_gam_fixed_2R])
pr_hets_2R = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_hyb_hets_2R])

count_SNP_df_2R = pr.count_overlaps({'SNPs': pr_gam_fixed_2R, 'selected_hets': pr_hets_2R}, features=ann_db_2R).df
count_SNP_df_2R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets
0,AgamP4_2R,VEuPathDB,protein_coding_gene,24245442,24251444,.,+,.,AGAP002619,unspecified product,NaN,NaN,NaN,NaN,NaN,0,0
1,AgamP4_2R,VEuPathDB,mRNA,24245442,24251444,.,+,.,AGAP002619-RA,unspecified product,AGAP002619,NaN,NaN,NaN,NaN,0,0
2,AgamP4_2R,VEuPathDB,exon,24245442,24245526,.,+,.,exon_AGAP002619-RA-E1,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,0,0
3,AgamP4_2R,VEuPathDB,exon,24245644,24246259,.,+,.,exon_AGAP002619-RA-E2,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,0,0
4,AgamP4_2R,VEuPathDB,exon,24247851,24249256,.,+,.,exon_AGAP002619-RA-E3,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53185,AgamP4_2R,VEuPathDB,exon,47020993,47023594,.,-,.,exon_AGAP029680-RA-E1,NaN,AGAP029680-RA,AGAP029680,NaN,NaN,NaN,0,0
53186,AgamP4_2R,VEuPathDB,CDS,47018304,47020467,.,-,0,AGAP029680-PA-CDS2,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,0,0
53187,AgamP4_2R,VEuPathDB,CDS,47020993,47021209,.,-,0,AGAP029680-PA-CDS1,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,0,0
53188,AgamP4_2R,VEuPathDB,three_prime_UTR,47017982,47018304,.,-,.,utr_AGAP029680-RA_1,NaN,AGAP029680-RA,NaN,NaN,NaN,NaN,0,0


In [20]:
#ratios

count_SNP_df_2R['selected_ratio'] = count_SNP_df_2R['selected_hets'] / count_SNP_df_2R['SNPs']
count_SNP_df_2R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets,selected_ratio
0,AgamP4_2R,VEuPathDB,protein_coding_gene,24245442,24251444,.,+,.,AGAP002619,unspecified product,NaN,NaN,NaN,NaN,NaN,0,0,NaN
1,AgamP4_2R,VEuPathDB,mRNA,24245442,24251444,.,+,.,AGAP002619-RA,unspecified product,AGAP002619,NaN,NaN,NaN,NaN,0,0,NaN
2,AgamP4_2R,VEuPathDB,exon,24245442,24245526,.,+,.,exon_AGAP002619-RA-E1,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,0,0,NaN
3,AgamP4_2R,VEuPathDB,exon,24245644,24246259,.,+,.,exon_AGAP002619-RA-E2,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,0,0,NaN
4,AgamP4_2R,VEuPathDB,exon,24247851,24249256,.,+,.,exon_AGAP002619-RA-E3,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53185,AgamP4_2R,VEuPathDB,exon,47020993,47023594,.,-,.,exon_AGAP029680-RA-E1,NaN,AGAP029680-RA,AGAP029680,NaN,NaN,NaN,0,0,NaN
53186,AgamP4_2R,VEuPathDB,CDS,47018304,47020467,.,-,0,AGAP029680-PA-CDS2,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,0,0,NaN
53187,AgamP4_2R,VEuPathDB,CDS,47020993,47021209,.,-,0,AGAP029680-PA-CDS1,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,0,0,NaN
53188,AgamP4_2R,VEuPathDB,three_prime_UTR,47017982,47018304,.,-,.,utr_AGAP029680-RA_1,NaN,AGAP029680-RA,NaN,NaN,NaN,NaN,0,0,NaN


In [21]:
grouped_2R = count_SNP_df_2R.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2R = grouped_2R[['gene_id', 'SNPs', 'selected_hets']]
grouped_2R['ratio'] = grouped_2R['selected_hets'] / grouped_2R['SNPs']
grouped_2R.sort_values('ratio', ascending=False)

,gene_id,SNPs,selected_hets,ratio
3523,AGAP013395,1,1,1.0
1103,AGAP002331,1,1,1.0
949,AGAP002153,1,1,1.0
3558,AGAP013455,1,1,1.0
1237,AGAP002472,1,1,1.0
...,...,...,...,...
3795,AGAP029806,0,0,NaN
3797,AGAP029862,0,0,NaN
3801,AGAP029915,0,0,NaN
3802,AGAP029916,0,0,NaN


In [ ]:
grouped_2R.to_csv("Snps_hets_3L.csv")

In [ ]:
#Scenario 2: het hybs with mask_ara_fixed_2R

mask_hets_2R_2=hyb_het_2R & mask_ara_fixed_2R

In [ ]:
#positions

pos_2R = allel.SortedIndex(ds2['variant_position'])

pos_ara_fixed_2R_2 = pos_2R[mask_ara_fixed_2R]

pos_hyb_hets_2R_2 = pos_2R[mask_hets_2R_2]

In [ ]:
import pyranges as pr

pr_ara_fixed_2R_2 = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_ara_fixed_2R_2])
pr_hets_2R_2 = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_hyb_hets_2R_2])

count_SNP_df_2R_2 = pr.count_overlaps({'SNPs': pr_ara_fixed_2R_2, 'selected_hets': pr_hets_2R_2}, features=ann_db_2R).df
count_SNP_df_2R_2

In [ ]:
count_SNP_df_2R_2['selected_ratio'] = count_SNP_df_2R_2['selected_hets'] / count_SNP_df_2R_2['SNPs']
count_SNP_df_2R_2

In [ ]:
grouped_2R_2 = count_SNP_df_2R_2.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2R_2 = grouped_2R_2[['gene_id', 'SNPs', 'selected_hets']]
grouped_2R_2 ['ratio'] = grouped_2R_2['selected_hets'] / grouped_2R_2['SNPs']
grouped_2R_2.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 3: Pure hybs 1/1 & gam 1/1 & ara -/- | 0/0

In [ ]:
#pure hyb hom alt
hyb_hom_alt_2R=g_2R[:,sample_is_hyb_2R].is_hom_alt().sum(axis=1) > 2

In [ ]:
#1/1 hybs and mask_gam_fixed_2R mask
mask_hom_2R_g=hyb_hom_alt_2R & mask_gam_fixed_2R

In [ ]:
#positions

pos_2R = allel.SortedIndex(ds2['variant_position'])

pos_gam_fixed_2R = pos_2R[mask_gam_fixed_2R]

pos_hyb_hom_2R_g = pos_2R[mask_hom_2R_g]

In [ ]:
import pyranges as pr

pr_gam_fixed_2R = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_gam_fixed_2R])
pr_hyb_hom_2R_g = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_hyb_hom_2R_g])

count_SNP_df_2R_g = pr.count_overlaps({'SNPs': pr_gam_fixed_2R, 'selected_hets': pr_hyb_hom_2R_g}, features=ann_db_2R).df
count_SNP_df_2R_g

In [ ]:
#ratios

count_SNP_df_2R_g['selected_ratio'] = count_SNP_df_2R_g['selected_hets'] / count_SNP_df_2R_g['SNPs']
count_SNP_df_2R_g

In [ ]:
grouped_2R_g = count_SNP_df_2R_g.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2R_g = grouped_2R_g[['gene_id', 'SNPs', 'selected_hets']]
grouped_2R_g['ratio'] = grouped_2R_g['selected_hets'] / grouped_2R_g['SNPs']
grouped_2R_g.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 4: 0/0 | -/- hybs, 1/1 ara, 0/0 | -/- gam
hyb_hom_missing_2R=g_2R[:,sample_is_hyb_2R].is_hom_alt().sum(axis=1) > 2
hyb_hom_hom_ref_2R=g_2R[:,sample_is_hyb_2R].is_hom_ref().sum(axis=1) > 2

hyb_total=hyb_hom_missing_2R | hyb_hom_hom_ref_2R

In [ ]:
#0/0 | -/- hybs and mask_ara_fixed_2R mask
mask_hom_ref_2R=hyb_total & mask_ara_fixed_2R

In [ ]:
#positions
pos_2R = allel.SortedIndex(ds2['variant_position'])

pos_ara_fixed_2R= pos_2R[mask_ara_fixed_2R]

pos_hyb_hom_ref_2R = pos_2R[mask_hom_ref_2R]

In [ ]:
import pyranges as pr

pr_ara_fixed_2R_homs = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_ara_fixed_2R])
pr_hyb_hom_ref_2R = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_hyb_hom_ref_2R])

count_SNP_df_2R_homs_ref = pr.count_overlaps({'SNPs': pr_ara_fixed_2R_homs, 'selected_hets': pr_hyb_hom_ref_2R}, features=ann_db_2R).df
count_SNP_df_2R_homs_ref

In [ ]:
#ratios
count_SNP_df_2R_homs_ref['selected_ratio'] = count_SNP_df_2R_homs_ref['selected_hets'] / count_SNP_df_2R_homs_ref['SNPs']
count_SNP_df_2R_homs_ref

In [ ]:
grouped_2R_ara_ref = count_SNP_df_2R_homs_ref.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2R_ara_ref = grouped_2R_ara_ref[['gene_id', 'SNPs', 'selected_hets']]
grouped_2R_ara_ref['ratio'] = grouped_2R_ara_ref['selected_hets'] / grouped_2R_ara_ref['SNPs']
grouped_2R_ara_ref.sort_values('ratio', ascending=False)

In [ ]:
#2L analysis

ds1 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2L.zarr')
ds1

ds1['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds1['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds1['sample_cohort_name']

sample_is_hyb_2L = ds1['sample_cohort_name'] == 'hyb'

sample_is_ara_2L = ds1['sample_cohort_name'] == 'ara'

sample_is_gam_2L = ds1['sample_cohort_name'] == 'gam'

g_2L = allel.GenotypeArray(ds1['call_genotype'])

In [ ]:
import pyranges as pr

# import annotations
ann_db_2L = pr.read_gff3("/Users/christinagkertsou/Desktop/Finalyearproject/VectorBase-56_AgambiaePEST.gff")
ann_db_2L = ann_db_2L['AgamP4_2L'] 
ann_db_2L

In [ ]:
#ara preparation

is_ara_missing_2L = g_2L[:,sample_is_ara_2L].is_missing().all(axis=1)

# is_ara_hom_alt = g[:,sample_is_ara].is_hom_alt().sum(axis=1) > 2
is_ara_hom_alt_2L = g_2L[:,sample_is_ara_2L].is_hom_alt().all(axis=1)
is_ara_hom_ref_2L = g_2L[:,sample_is_ara_2L].is_hom_ref().sum(axis=1) == 2

In [ ]:
#gam preparation

is_gam_missing_2L = g_2L[:,sample_is_gam_2L].is_missing().sum(axis=1) > 6
is_gam_hom_alt_2L = g_2L[:,sample_is_gam_2L].is_hom_alt().sum(axis=1) > 6
is_gam_hom_ref_2L = g_2L[:,sample_is_gam_2L].is_hom_ref().sum(axis=1) > 6

In [ ]:
#hyribrids hets

hyb_het_2L = g_2L[:,sample_is_hyb_2L].is_het().sum(axis=1) > 2

#The ara and gam fixed masks

mask_ara_fixed_2L = is_ara_hom_alt_2L & (is_gam_missing_2L | is_gam_hom_ref_2L)

mask_gam_fixed_2L = is_gam_hom_alt_2L & (is_ara_missing_2L | is_ara_hom_ref_2L)

In [ ]:
#Scenario 1: het hybs with mask_gam_fixed_2L

mask_hets_2L=hyb_het_2L & mask_gam_fixed_2L

#positions

pos_2L = allel.SortedIndex(ds1['variant_position'])

pos_gam_fixed_2L = pos_2L[mask_gam_fixed_2L]

pos_hyb_hets_2L = pos_2L[mask_hets_2L]

In [ ]:
import pyranges as pr

pr_gam_fixed_2L = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_gam_fixed_2L])
pr_hets_2L = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_hyb_hets_2L])

count_SNP_df_2L = pr.count_overlaps({'SNPs': pr_gam_fixed_2L, 'selected_hets': pr_hets_2L}, features=ann_db_2L).df
count_SNP_df_2L

In [ ]:
#ratios

count_SNP_df_2L['selected_ratio'] = count_SNP_df_2L['selected_hets'] / count_SNP_df_2L['SNPs']
count_SNP_df_2L

In [ ]:
grouped_2L = count_SNP_df_2L.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2L = grouped_2L[['gene_id', 'SNPs', 'selected_hets']]
grouped_2L['ratio'] = grouped_2L['selected_hets'] / grouped_2L['SNPs']
grouped_2L.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 2: het hybs with mask_ara_fixed_2L

mask_hets_2L_2=hyb_het_2L & mask_ara_fixed_2L

#positions

pos_2L = allel.SortedIndex(ds1['variant_position'])

pos_ara_fixed_2L_2 = pos_2L[mask_ara_fixed_2L]

pos_hyb_hets_2L_2 = pos_2L[mask_hets_2L_2]

In [ ]:
import pyranges as pr

pr_ara_fixed_2L_2 = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_ara_fixed_2L_2])
pr_hets_2L_2 = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_hyb_hets_2L_2])

count_SNP_df_2L_2 = pr.count_overlaps({'SNPs': pr_ara_fixed_2L_2, 'selected_hets': pr_hets_2L_2}, features=ann_db_2L).df
count_SNP_df_2L_2

In [ ]:
count_SNP_df_2L_2['selected_ratio'] = count_SNP_df_2L_2['selected_hets'] / count_SNP_df_2L_2['SNPs']
count_SNP_df_2L_2

In [ ]:
grouped_2L_2 = count_SNP_df_2L_2.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2L_2 = grouped_2L_2[['gene_id', 'SNPs', 'selected_hets']]
grouped_2L_2 ['ratio'] = grouped_2L_2['selected_hets'] / grouped_2L_2['SNPs']
grouped_2L_2.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 3: Pure hybs 1/1 & gam 1/1 & ara -/- | 0/0

#pure hyb hom alt
hyb_hom_alt_2L=g_2L[:,sample_is_hyb_2L].is_hom_alt().sum(axis=1) > 2

#1/1 hybs and mask_gam_fixed_2L mask
mask_hom_2L_g=hyb_hom_alt_2L & mask_gam_fixed_2L

#positions

pos_2L = allel.SortedIndex(ds1['variant_position'])

pos_gam_fixed_2L = pos_2L[mask_gam_fixed_2L]

pos_hyb_hom_2L_g = pos_2L[mask_hom_2L_g]

In [ ]:
import pyranges as pr

pr_gam_fixed_2L = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_gam_fixed_2L])
pr_hyb_hom_2L_g = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_hyb_hom_2L_g])

count_SNP_df_2L_g = pr.count_overlaps({'SNPs': pr_gam_fixed_2L, 'selected_hets': pr_hyb_hom_2L_g}, features=ann_db_2L).df
count_SNP_df_2L_g

In [ ]:
#ratios

count_SNP_df_2L_g['selected_ratio'] = count_SNP_df_2L_g['selected_hets'] / count_SNP_df_2L_g['SNPs']
count_SNP_df_2L_g

In [ ]:
grouped_2L_g = count_SNP_df_2L_g.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2L_g = grouped_2L_g[['gene_id', 'SNPs', 'selected_hets']]
grouped_2L_g['ratio'] = grouped_2L_g['selected_hets'] / grouped_2L_g['SNPs']
grouped_2L_g.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 4: 0/0 | -/- hybs, 1/1 ara, 0/0 | -/- gam
hyb_hom_missing_2L=g_2L[:,sample_is_hyb_2L].is_hom_alt().sum(axis=1) > 2
hyb_hom_hom_ref_2L=g_2L[:,sample_is_hyb_2L].is_hom_ref().sum(axis=1) > 2

hyb_total=hyb_hom_missing_2L | hyb_hom_hom_ref_2L

#0/0 | -/- hybs and mask_ara_fixed_2L mask
mask_hom_ref_2L=hyb_total & mask_ara_fixed_2L

#positions
pos_2L = allel.SortedIndex(ds1['variant_position'])

pos_ara_fixed_2L= pos_2L[mask_ara_fixed_2L]

pos_hyb_hom_ref_2L = pos_2L[mask_hom_ref_2L]

In [ ]:
import pyranges as pr

pr_ara_fixed_2L_homs = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_ara_fixed_2L])
pr_hyb_hom_ref_2L = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_hyb_hom_ref_2L])

count_SNP_df_2L_homs_ref = pr.count_overlaps({'SNPs': pr_ara_fixed_2L_homs, 'selected_hets': pr_hyb_hom_ref_2L}, features=ann_db_2L).df
count_SNP_df_2L_homs_ref

In [ ]:
#ratios
count_SNP_df_2L_homs_ref['selected_ratio'] = count_SNP_df_2L_homs_ref['selected_hets'] / count_SNP_df_2L_homs_ref['SNPs']
count_SNP_df_2L_homs_ref

In [ ]:
grouped_2L_ara_ref = count_SNP_df_2L_homs_ref.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2L_ara_ref = grouped_2L_ara_ref[['gene_id', 'SNPs', 'selected_hets']]
grouped_2L_ara_ref['ratio'] = grouped_2L_ara_ref['selected_hets'] / grouped_2L_ara_ref['SNPs']
grouped_2L_ara_ref.sort_values('ratio', ascending=False)

In [ ]:
grouped_2L.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/2L1.csv")
grouped_2L_2.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/2L2.csv")
grouped_2L_g.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/2L3.csv")
grouped_2L_ara_ref.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/2L4.csv")

In [ ]:
#3L analysis

ds3 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3L.zarr')
ds3

ds3['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds3['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds3['sample_cohort_name']

sample_is_hyb_3L = ds3['sample_cohort_name'] == 'hyb'

sample_is_ara_3L = ds3['sample_cohort_name'] == 'ara'

sample_is_gam_3L = ds3['sample_cohort_name'] == 'gam'

g_3L = allel.GenotypeArray(ds3['call_genotype'])

In [ ]:
import pyranges as pr

# import annotations
ann_db_3L = pr.read_gff3("/Users/christinagkertsou/Desktop/Finalyearproject/VectorBase-56_AgambiaePEST.gff")
ann_db_3L = ann_db_3L['AgamP4_3L'] 
ann_db_3L

In [ ]:
#ara preparation

is_ara_missing_3L = g_3L[:,sample_is_ara_3L].is_missing().all(axis=1)

# is_ara_hom_alt = g[:,sample_is_ara].is_hom_alt().sum(axis=1) > 2
is_ara_hom_alt_3L = g_3L[:,sample_is_ara_3L].is_hom_alt().all(axis=1)
is_ara_hom_ref_3L = g_3L[:,sample_is_ara_3L].is_hom_ref().sum(axis=1) == 2

In [ ]:
#gam preparation

is_gam_missing_3L = g_3L[:,sample_is_gam_3L].is_missing().sum(axis=1) > 6
is_gam_hom_alt_3L = g_3L[:,sample_is_gam_3L].is_hom_alt().sum(axis=1) > 6
is_gam_hom_ref_3L = g_3L[:,sample_is_gam_3L].is_hom_ref().sum(axis=1) > 6

In [ ]:
#hyribrids hets

hyb_het_3L = g_3L[:,sample_is_hyb_3L].is_het().sum(axis=1) > 2

#The ara and gam fixed masks

mask_ara_fixed_3L = is_ara_hom_alt_3L & (is_gam_missing_3L | is_gam_hom_ref_3L)

mask_gam_fixed_3L = is_gam_hom_alt_3L & (is_ara_missing_3L | is_ara_hom_ref_3L)

In [ ]:
#Scenario 1: het hybs with mask_gam_fixed_3L

mask_hets_3L=hyb_het_3L & mask_gam_fixed_3L

#positions

pos_3L = allel.SortedIndex(ds3['variant_position'])

pos_gam_fixed_3L = pos_3L[mask_gam_fixed_3L]

pos_hyb_hets_3L = pos_3L[mask_hets_3L]

In [ ]:
import pyranges as pr

pr_gam_fixed_3L = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_gam_fixed_3L])
pr_hets_3L = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_hyb_hets_3L])

count_SNP_df_3L = pr.count_overlaps({'SNPs': pr_gam_fixed_3L, 'selected_hets': pr_hets_3L}, features=ann_db_3L).df
count_SNP_df_3L

In [ ]:
#ratios

count_SNP_df_3L['selected_ratio'] = count_SNP_df_3L['selected_hets'] / count_SNP_df_3L['SNPs']
count_SNP_df_3L

In [ ]:
grouped_3L = count_SNP_df_3L.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3L = grouped_3L[['gene_id', 'SNPs', 'selected_hets']]
grouped_3L['ratio'] = grouped_3L['selected_hets'] / grouped_3L['SNPs']
grouped_3L.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 2: het hybs with mask_ara_fixed_3L

mask_hets_3L_2=hyb_het_3L & mask_ara_fixed_3L

#positions

pos_3L = allel.SortedIndex(ds3['variant_position'])

pos_ara_fixed_3L_2 = pos_3L[mask_ara_fixed_3L]

pos_hyb_hets_3L_2 = pos_3L[mask_hets_3L_2]

In [ ]:
import pyranges as pr

pr_ara_fixed_3L_2 = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_ara_fixed_3L_2])
pr_hets_3L_2 = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_hyb_hets_3L_2])

count_SNP_df_3L_2 = pr.count_overlaps({'SNPs': pr_ara_fixed_3L_2, 'selected_hets': pr_hets_3L_2}, features=ann_db_3L).df
count_SNP_df_3L_2

In [ ]:
count_SNP_df_3L_2['selected_ratio'] = count_SNP_df_3L_2['selected_hets'] / count_SNP_df_3L_2['SNPs']
count_SNP_df_3L_2

In [ ]:
grouped_3L_2 = count_SNP_df_3L_2.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3L_2 = grouped_3L_2[['gene_id', 'SNPs', 'selected_hets']]
grouped_3L_2 ['ratio'] = grouped_3L_2['selected_hets'] / grouped_3L_2['SNPs']
grouped_3L_2.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 3: Pure hybs 1/1 & gam 1/1 & ara -/- | 0/0

#pure hyb hom alt
hyb_hom_alt_3L=g_3L[:,sample_is_hyb_3L].is_hom_alt().sum(axis=1) > 2

#1/1 hybs and mask_gam_fixed_3L mask
mask_hom_3L_g=hyb_hom_alt_3L & mask_gam_fixed_3L

#positions

pos_3L = allel.SortedIndex(ds3['variant_position'])

pos_gam_fixed_3L = pos_3L[mask_gam_fixed_3L]

pos_hyb_hom_3L_g = pos_3L[mask_hom_3L_g]

In [ ]:
import pyranges as pr

pr_gam_fixed_3L = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_gam_fixed_3L])
pr_hyb_hom_3L_g = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_hyb_hom_3L_g])

count_SNP_df_3L_g = pr.count_overlaps({'SNPs': pr_gam_fixed_3L, 'selected_hets': pr_hyb_hom_3L_g}, features=ann_db_3L).df
count_SNP_df_3L_g

In [ ]:
#ratios

count_SNP_df_3L_g['selected_ratio'] = count_SNP_df_3L_g['selected_hets'] / count_SNP_df_3L_g['SNPs']
count_SNP_df_3L_g

In [ ]:
grouped_3L_g = count_SNP_df_3L_g.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3L_g = grouped_3L_g[['gene_id', 'SNPs', 'selected_hets']]
grouped_3L_g['ratio'] = grouped_3L_g['selected_hets'] / grouped_3L_g['SNPs']
grouped_3L_g.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 4: 0/0 | -/- hybs, 1/1 ara, 0/0 | -/- gam
hyb_hom_missing_3L=g_3L[:,sample_is_hyb_3L].is_hom_alt().sum(axis=1) > 2
hyb_hom_hom_ref_3L=g_3L[:,sample_is_hyb_3L].is_hom_ref().sum(axis=1) > 2

hyb_total=hyb_hom_missing_3L | hyb_hom_hom_ref_3L

#0/0 | -/- hybs and mask_ara_fixed_3L mask
mask_hom_ref_3L=hyb_total & mask_ara_fixed_3L

#positions
pos_3L = allel.SortedIndex(ds3['variant_position'])

pos_ara_fixed_3L= pos_3L[mask_ara_fixed_3L]

pos_hyb_hom_ref_3L = pos_3L[mask_hom_ref_3L]

In [ ]:
import pyranges as pr

pr_ara_fixed_3L_homs = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_ara_fixed_3L])
pr_hyb_hom_ref_3L = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_hyb_hom_ref_3L])

count_SNP_df_3L_homs_ref = pr.count_overlaps({'SNPs': pr_ara_fixed_3L_homs, 'selected_hets': pr_hyb_hom_ref_3L}, features=ann_db_3L).df
count_SNP_df_3L_homs_ref

In [ ]:
#ratios
count_SNP_df_3L_homs_ref['selected_ratio'] = count_SNP_df_3L_homs_ref['selected_hets'] / count_SNP_df_3L_homs_ref['SNPs']
count_SNP_df_3L_homs_ref

In [ ]:
grouped_3L_ara_ref = count_SNP_df_3L_homs_ref.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3L_ara_ref = grouped_3L_ara_ref[['gene_id', 'SNPs', 'selected_hets']]
grouped_3L_ara_ref['ratio'] = grouped_3L_ara_ref['selected_hets'] / grouped_3L_ara_ref['SNPs']
grouped_3L_ara_ref.sort_values('ratio', ascending=False)

In [ ]:
grouped_3L.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3L1.csv")
grouped_3L_2.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3L2.csv")
grouped_3L_g.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3L3.csv")
grouped_3L_ara_ref.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3L4.csv")

In [ ]:
#3R analysis
ds4 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3R.zarr')
ds4

ds4['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds4['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds4['sample_cohort_name']

sample_is_hyb_3R = ds4['sample_cohort_name'] == 'hyb'

sample_is_ara_3R = ds4['sample_cohort_name'] == 'ara'

sample_is_gam_3R = ds4['sample_cohort_name'] == 'gam'

g_3R = allel.GenotypeArray(ds4['call_genotype'])

In [ ]:
import pyranges as pr

# import annotations
ann_db_3R = pr.read_gff3("/Users/christinagkertsou/Desktop/Finalyearproject/VectorBase-56_AgambiaePEST.gff")
ann_db_3R = ann_db_3R['AgamP4_3R'] 
ann_db_3R

In [ ]:
#ara preparation

is_ara_missing_3R = g_3R[:,sample_is_ara_3R].is_missing().all(axis=1)

# is_ara_hom_alt = g[:,sample_is_ara].is_hom_alt().sum(axis=1) > 2
is_ara_hom_alt_3R = g_3R[:,sample_is_ara_3R].is_hom_alt().all(axis=1)
is_ara_hom_ref_3R = g_3R[:,sample_is_ara_3R].is_hom_ref().sum(axis=1) == 2

In [ ]:
#gam preparation

is_gam_missing_3R = g_3R[:,sample_is_gam_3R].is_missing().sum(axis=1) > 6
is_gam_hom_alt_3R = g_3R[:,sample_is_gam_3R].is_hom_alt().sum(axis=1) > 6
is_gam_hom_ref_3R = g_3R[:,sample_is_gam_3R].is_hom_ref().sum(axis=1) > 6

In [ ]:
#hyribrids hets

hyb_het_3R = g_3R[:,sample_is_hyb_3R].is_het().sum(axis=1) > 2

#The ara and gam fixed masks

mask_ara_fixed_3R = is_ara_hom_alt_3R & (is_gam_missing_3R | is_gam_hom_ref_3R)

mask_gam_fixed_3R = is_gam_hom_alt_3R & (is_ara_missing_3R | is_ara_hom_ref_3R)

In [ ]:
#Scenario 1: het hybs with mask_gam_fixed_3R

mask_hets_3R=hyb_het_3R & mask_gam_fixed_3R

#positions

pos_3R = allel.SortedIndex(ds4['variant_position'])

pos_gam_fixed_3R = pos_3R[mask_gam_fixed_3R]

pos_hyb_hets_3R = pos_3R[mask_hets_3R]

In [ ]:
import pyranges as pr

pr_gam_fixed_3R = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_gam_fixed_3R])
pr_hets_3R = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_hyb_hets_3R])

count_SNP_df_3R = pr.count_overlaps({'SNPs': pr_gam_fixed_3R, 'selected_hets': pr_hets_3R}, features=ann_db_3R).df
count_SNP_df_3R

In [ ]:
#ratios

count_SNP_df_3R['selected_ratio'] = count_SNP_df_3R['selected_hets'] / count_SNP_df_3R['SNPs']
count_SNP_df_3R

In [ ]:
grouped_3R = count_SNP_df_3R.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3R = grouped_3R[['gene_id', 'SNPs', 'selected_hets']]
grouped_3R['ratio'] = grouped_3R['selected_hets'] / grouped_3R['SNPs']
grouped_3R.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 2: het hybs with mask_ara_fixed_3R

mask_hets_3R_2=hyb_het_3R & mask_ara_fixed_3R

#positions

pos_3R = allel.SortedIndex(ds4['variant_position'])

pos_ara_fixed_3R_2 = pos_3R[mask_ara_fixed_3R]

pos_hyb_hets_3R_2 = pos_3R[mask_hets_3R_2]

In [ ]:
import pyranges as pr

pr_ara_fixed_3R_2 = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_ara_fixed_3R_2])
pr_hets_3R_2 = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_hyb_hets_3R_2])

count_SNP_df_3R_2 = pr.count_overlaps({'SNPs': pr_ara_fixed_3R_2, 'selected_hets': pr_hets_3R_2}, features=ann_db_3R).df
count_SNP_df_3R_2

In [ ]:
count_SNP_df_3R_2['selected_ratio'] = count_SNP_df_3R_2['selected_hets'] / count_SNP_df_3R_2['SNPs']
count_SNP_df_3R_2

In [ ]:
grouped_3R_2 = count_SNP_df_3R_2.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3R_2 = grouped_3R_2[['gene_id', 'SNPs', 'selected_hets']]
grouped_3R_2 ['ratio'] = grouped_3R_2['selected_hets'] / grouped_3R_2['SNPs']
grouped_3R_2.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 3: Pure hybs 1/1 & gam 1/1 & ara -/- | 0/0

#pure hyb hom alt
hyb_hom_alt_3R=g_3R[:,sample_is_hyb_3R].is_hom_alt().sum(axis=1) > 2

#1/1 hybs and mask_gam_fixed_3R mask
mask_hom_3R_g=hyb_hom_alt_3R & mask_gam_fixed_3R

#positions

pos_3R = allel.SortedIndex(ds4['variant_position'])

pos_gam_fixed_3R = pos_3R[mask_gam_fixed_3R]

pos_hyb_hom_3R_g = pos_3R[mask_hom_3R_g]

In [ ]:
import pyranges as pr

pr_gam_fixed_3R = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_gam_fixed_3R])
pr_hyb_hom_3R_g = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_hyb_hom_3R_g])

count_SNP_df_3R_g = pr.count_overlaps({'SNPs': pr_gam_fixed_3R, 'selected_hets': pr_hyb_hom_3R_g}, features=ann_db_3R).df
count_SNP_df_3R_g

In [ ]:
#ratios

count_SNP_df_3R_g['selected_ratio'] = count_SNP_df_3R_g['selected_hets'] / count_SNP_df_3R_g['SNPs']
count_SNP_df_3R_g

In [ ]:
grouped_3R_g = count_SNP_df_3R_g.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3R_g = grouped_3R_g[['gene_id', 'SNPs', 'selected_hets']]
grouped_3R_g['ratio'] = grouped_3R_g['selected_hets'] / grouped_3R_g['SNPs']
grouped_3R_g.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 4: 0/0 | -/- hybs, 1/1 ara, 0/0 | -/- gam
hyb_hom_missing_3R=g_3R[:,sample_is_hyb_3R].is_hom_alt().sum(axis=1) > 2
hyb_hom_hom_ref_3R=g_3R[:,sample_is_hyb_3R].is_hom_ref().sum(axis=1) > 2

hyb_total=hyb_hom_missing_3R | hyb_hom_hom_ref_3R

#0/0 | -/- hybs and mask_ara_fixed_3R mask
mask_hom_ref_3R=hyb_total & mask_ara_fixed_3R

#positions
pos_3R = allel.SortedIndex(ds4['variant_position'])

pos_ara_fixed_3R= pos_3R[mask_ara_fixed_3R]

pos_hyb_hom_ref_3R = pos_3R[mask_hom_ref_3R]

In [ ]:
import pyranges as pr

pr_ara_fixed_3R_homs = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_ara_fixed_3R])
pr_hyb_hom_ref_3R = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_hyb_hom_ref_3R])

count_SNP_df_3R_homs_ref = pr.count_overlaps({'SNPs': pr_ara_fixed_3R_homs, 'selected_hets': pr_hyb_hom_ref_3R}, features=ann_db_3R).df
count_SNP_df_3R_homs_ref

In [ ]:
#ratios
count_SNP_df_3R_homs_ref['selected_ratio'] = count_SNP_df_3R_homs_ref['selected_hets'] / count_SNP_df_3R_homs_ref['SNPs']
count_SNP_df_3R_homs_ref

In [ ]:
grouped_3R_ara_ref = count_SNP_df_3R_homs_ref.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3R_ara_ref = grouped_3R_ara_ref[['gene_id', 'SNPs', 'selected_hets']]
grouped_3R_ara_ref['ratio'] = grouped_3R_ara_ref['selected_hets'] / grouped_3R_ara_ref['SNPs']
grouped_3R_ara_ref.sort_values('ratio', ascending=False)

In [ ]:
grouped_3R.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3R1.csv")
grouped_3R_2.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3R2.csv")
grouped_3R_g.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3R3.csv")
grouped_3R_ara_ref.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/3R4.csv")

In [ ]:
#X analysis

ds5 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_X.zarr')
ds5

ds5['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds5['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds5['sample_cohort_name']

sample_is_hyb_X = ds5['sample_cohort_name'] == 'hyb'

sample_is_ara_X = ds5['sample_cohort_name'] == 'ara'

sample_is_gam_X = ds5['sample_cohort_name'] == 'gam'

g_X = allel.GenotypeArray(ds5['call_genotype'])

In [ ]:
import pyranges as pr

# import annotations
ann_db_X = pr.read_gff3("/Users/christinagkertsou/Desktop/Finalyearproject/VectorBase-56_AgambiaePEST.gff")
ann_db_X = ann_db_X['AgamP4_X'] 
ann_db_X

In [ ]:
#ara preparation

is_ara_missing_X = g_X[:,sample_is_ara_X].is_missing().all(axis=1)

# is_ara_hom_alt = g[:,sample_is_ara].is_hom_alt().sum(axis=1) > 2
is_ara_hom_alt_X = g_X[:,sample_is_ara_X].is_hom_alt().all(axis=1)
is_ara_hom_ref_X = g_X[:,sample_is_ara_X].is_hom_ref().sum(axis=1) == 2

In [ ]:
#gam preparation

is_gam_missing_X = g_X[:,sample_is_gam_X].is_missing().sum(axis=1) > 6
is_gam_hom_alt_X = g_X[:,sample_is_gam_X].is_hom_alt().sum(axis=1) > 6
is_gam_hom_ref_X = g_X[:,sample_is_gam_X].is_hom_ref().sum(axis=1) > 6

In [ ]:
#hyribrids hets

hyb_het_X = g_X[:,sample_is_hyb_X].is_het().sum(axis=1) > 2

#The ara and gam fixed masks

mask_ara_fixed_X = is_ara_hom_alt_X & (is_gam_missing_X | is_gam_hom_ref_X)

mask_gam_fixed_X = is_gam_hom_alt_X & (is_ara_missing_X | is_ara_hom_ref_X)

In [ ]:
#Scenario 1: het hybs with mask_gam_fixed_X

mask_hets_X=hyb_het_X & mask_gam_fixed_X

#positions

pos_X = allel.SortedIndex(ds5['variant_position'])

pos_gam_fixed_X = pos_X[mask_gam_fixed_X]

pos_hyb_hets_X = pos_X[mask_hets_X]

In [ ]:
import pyranges as pr

pr_gam_fixed_X = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_gam_fixed_X])
pr_hets_X = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_hyb_hets_X])

count_SNP_df_X = pr.count_overlaps({'SNPs': pr_gam_fixed_X, 'selected_hets': pr_hets_X}, features=ann_db_X).df
count_SNP_df_X

In [ ]:
#ratios

count_SNP_df_X['selected_ratio'] = count_SNP_df_X['selected_hets'] / count_SNP_df_X['SNPs']
count_SNP_df_X

In [ ]:
grouped_X = count_SNP_df_X.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_X = grouped_X[['gene_id', 'SNPs', 'selected_hets']]
grouped_X['ratio'] = grouped_X['selected_hets'] / grouped_X['SNPs']

In [ ]:
#Scenario 2: het hybs with mask_ara_fixed_X

mask_hets_X_2=hyb_het_X & mask_ara_fixed_X

#positions

pos_X = allel.SortedIndex(ds5['variant_position'])

pos_ara_fixed_X_2 = pos_X[mask_ara_fixed_X]

pos_hyb_hets_X_2 = pos_X[mask_hets_X_2]

In [ ]:
import pyranges as pr

pr_ara_fixed_X_2 = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_ara_fixed_X_2])
pr_hets_X_2 = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_hyb_hets_X_2])

count_SNP_df_X_2 = pr.count_overlaps({'SNPs': pr_ara_fixed_X_2, 'selected_hets': pr_hets_X_2}, features=ann_db_X).df
count_SNP_df_X_2

In [ ]:
count_SNP_df_X_2['selected_ratio'] = count_SNP_df_X_2['selected_hets'] / count_SNP_df_X_2['SNPs']
count_SNP_df_X_2

In [ ]:
grouped_X_2 = count_SNP_df_X_2.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_X_2 = grouped_X_2[['gene_id', 'SNPs', 'selected_hets']]
grouped_X_2 ['ratio'] = grouped_X_2['selected_hets'] / grouped_X_2['SNPs']
grouped_X_2.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 3: Pure hybs 1/1 & gam 1/1 & ara -/- | 0/0

#pure hyb hom alt
hyb_hom_alt_X=g_X[:,sample_is_hyb_X].is_hom_alt().sum(axis=1) > 2

#1/1 hybs and mask_gam_fixed_X mask
mask_hom_X_g=hyb_hom_alt_X & mask_gam_fixed_X

#positions

pos_X = allel.SortedIndex(ds5['variant_position'])

pos_gam_fixed_X = pos_X[mask_gam_fixed_X]

pos_hyb_hom_X_g = pos_X[mask_hom_X_g]

In [ ]:
import pyranges as pr

pr_gam_fixed_X = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_gam_fixed_X])
pr_hyb_hom_X_g = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_hyb_hom_X_g])

count_SNP_df_X_g = pr.count_overlaps({'SNPs': pr_gam_fixed_X, 'selected_hets': pr_hyb_hom_X_g}, features=ann_db_X).df
count_SNP_df_X_g

In [ ]:
#ratios

count_SNP_df_X_g['selected_ratio'] = count_SNP_df_X_g['selected_hets'] / count_SNP_df_X_g['SNPs']
count_SNP_df_X_g

In [ ]:
grouped_X_g = count_SNP_df_X_g.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_X_g = grouped_X_g[['gene_id', 'SNPs', 'selected_hets']]
grouped_X_g['ratio'] = grouped_X_g['selected_hets'] / grouped_X_g['SNPs']
grouped_X_g.sort_values('ratio', ascending=False)

In [ ]:
#Scenario 4: 0/0 | -/- hybs, 1/1 ara, 0/0 | -/- gam
hyb_hom_missing_X=g_X[:,sample_is_hyb_X].is_hom_alt().sum(axis=1) > 2
hyb_hom_hom_ref_X=g_X[:,sample_is_hyb_X].is_hom_ref().sum(axis=1) > 2

hyb_total=hyb_hom_missing_X | hyb_hom_hom_ref_X

#0/0 | -/- hybs and mask_ara_fixed_X mask
mask_hom_ref_X=hyb_total & mask_ara_fixed_X

#positions
pos_X = allel.SortedIndex(ds5['variant_position'])

pos_ara_fixed_X= pos_X[mask_ara_fixed_X]

pos_hyb_hom_ref_X = pos_X[mask_hom_ref_X]

In [ ]:
import pyranges as pr

pr_ara_fixed_X_homs = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_ara_fixed_X])
pr_hyb_hom_ref_X = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_hyb_hom_ref_X])

count_SNP_df_X_homs_ref = pr.count_overlaps({'SNPs': pr_ara_fixed_X_homs, 'selected_hets': pr_hyb_hom_ref_X}, features=ann_db_X).df
count_SNP_df_X_homs_ref

In [ ]:
#ratios
count_SNP_df_X_homs_ref['selected_ratio'] = count_SNP_df_X_homs_ref['selected_hets'] / count_SNP_df_X_homs_ref['SNPs']
count_SNP_df_X_homs_ref

In [ ]:
grouped_X_ara_ref = count_SNP_df_X_homs_ref.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_X_ara_ref = grouped_X_ara_ref[['gene_id', 'SNPs', 'selected_hets']]
grouped_X_ara_ref['ratio'] = grouped_X_ara_ref['selected_hets'] / grouped_X_ara_ref['SNPs']
grouped_X_ara_ref.sort_values('ratio', ascending=False)

In [ ]:
grouped_X_2.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/X2.csv")
grouped_X_g.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/X3.csv")
grouped_X_ara_ref.to_csv("/Users/christinagkertsou/Desktop/ola_gambiae/X4.csv")